In [1]:
# Multithreading is used when you want to run multiple code simultaneously in single script
# If not using threading, code runs in the main thread
# when using it , threaded code runs in secondary thread

In [2]:
# Use cases :
# e.g. downloading resources parallely from internet
# e.g. performing concurrent tasks such as detecting people intrusion as well as directing security measures

In [3]:
# Python runs code one by one
# for concurrency we use multithreading

In Python's threading module, there is always one main thread, which is the initial thread that starts when you run a Python program. This main thread is responsible for creating additional threads and managing the overall flow of execution.

The other threads created using the threading module are considered secondary threads. These threads are used to perform tasks concurrently with the main thread. They can execute code independently and concurrently, allowing for parallelism in your program

In [4]:
import threading
import time

In [5]:
# indicates some task being done
def func(seconds):
    print(f'Sleeping for {seconds} seconds')
    time.sleep(seconds)
    print(f'{seconds} seconds ended')

In [6]:
func(4)
func(2)
func(1)

Sleeping for 4 seconds
4 seconds ended
Sleeping for 2 seconds
2 seconds ended
Sleeping for 1 seconds
1 seconds ended


In [7]:
threading.Thread(target=func,args=(4,)).start() # secondary thread
threading.Thread(target=func,args=(2,)).start() # secondary thread
threading.Thread(target=func,args=(1,)).start() # secondary thread
print('end') # main thread 

Sleeping for 4 seconds
Sleeping for 2 seconds
Sleeping for 1 seconds
end
1 seconds ended
2 seconds ended
4 seconds ended


In [8]:
# checking performance
time1 = time.perf_counter() # performance counter
func(5)
func(3)
func(2)
time2 = time.perf_counter()
print(time2-time1)

Sleeping for 5 seconds
5 seconds ended
Sleeping for 3 seconds
3 seconds ended
Sleeping for 2 seconds
2 seconds ended
10.003214399970602


In [9]:
# checking performance using threading
time1 = time.perf_counter()
threading.Thread(target=func,args=[5]).start() # secondary thread
threading.Thread(target=func,args=[3]).start() # secondary thread
threading.Thread(target=func,args=[2]).start() # secondary thread
time2 = time.perf_counter()
print(time2-time1)

Sleeping for 5 seconds
Sleeping for 3 seconds
Sleeping for 2 seconds
0.005616299982648343
2 seconds ended
3 seconds ended
5 seconds ended


The problem in the above cell is that the performance counter gave almost 0 seconds. It's because when we run the code,main thread and secondary threads gets executed at the same time. We can use that join method to prevent this. Because when we call join method on secondary thread, main thread code below that thread start calling, will not run until and unless that secondary thread is finished

In [10]:
# checking performance using threading again but now using join method
time1 = time.perf_counter()
thread1 = threading.Thread(target=func,args=[5])
thread2 = threading.Thread(target=func,args=[3])
thread3 = threading.Thread(target=func,args=[2])
thread1.start()
thread2.start()
thread3.start()
thread1.join()
time2 = time.perf_counter()
print(time2-time1)

Sleeping for 5 seconds
Sleeping for 3 seconds
Sleeping for 2 seconds
2 seconds ended
3 seconds ended
5 seconds ended
5.00721979996888


so in above cell we used join method on that specific thread which ends at the end, we could also use join method on all secondary threads which means python scripts have to wait till all the secondary threads functionality ends and then execute the main thread code below those secondary threads start calling

when trying to run multiple threads on same code with different arguments threadpoolexecutor is a go

In [11]:
from concurrent.futures import ThreadPoolExecutor

In [12]:
with ThreadPoolExecutor() as executor:
    future1 = executor.submit(func,5)
    future2 = executor.submit(func,3)
    future3 = executor.submit(func,2)
    print(future1.result())
    print(future2.result())
    print(future3.result())

Sleeping for 5 seconds
Sleeping for 3 seconds
Sleeping for 2 seconds
2 seconds ended
3 seconds ended
5 seconds ended
None
None
None


In [13]:
# you can do mapping as well using threadpoolexecutor
with ThreadPoolExecutor() as executor:
    l = [5,3,2]
    results = executor.map(func,l)
    print(results)

Sleeping for 5 seconds
Sleeping for 3 seconds
Sleeping for 2 seconds
<generator object Executor.map.<locals>.result_iterator at 0x00000225FE595740>
2 seconds ended
3 seconds ended
5 seconds ended


In [14]:
type(results)

generator

In [15]:
for result in results:
    print(result)

None
None
None


In [16]:
# but if you return some value it will return that value

In [17]:
def killer(seconds):
    print(f'Target is required to be killed in {seconds} seconds')
    time.sleep(seconds)
    if seconds > 3 :
        print(f'Target is killed in {seconds} seconds')
        return True
    return False

In [18]:
with ThreadPoolExecutor() as executor:
    l = [5,3,2]
    results = executor.map(killer,l)
    for result in results:
        print(result)

Target is required to be killed in 5 seconds
Target is required to be killed in 3 seconds
Target is required to be killed in 2 seconds
Target is killed in 5 seconds
True
False
False


In [19]:
# Running Synchronously means running everything in order (i.e. line by line code execution)

In [20]:
# CPU-bound tasks vs IO-bound tasks

In [21]:
# CPU-bound task --> Multiprocessing
# IO-bound task --> Multithreading

In [22]:
import time
import threading
import concurrent.futures

In [23]:
def do_something(seconds):
    print(f'sleeping for {seconds} seconds ')
    time.sleep(seconds)
    print(f'Done {seconds}')
    return f'result {seconds}'

In [24]:
t1 = time.perf_counter()
threads = []
for _ in range(10):  # running multiple threads using for loops 
    t = threading.Thread(target=do_something,args = [1])
    t.start()
    threads.append(t)
for thread in threads:
    thread.join()
print('end')
t2 = time.perf_counter()
print(t2-t1)

sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
end
1.012816800037399


In [25]:
# using list_comprehension for multiple threads although you can do same using map function

In [26]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(do_something,1) for _ in range(10)]
    for f in concurrent.futures.as_completed(results):
        print(f.result())

sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
sleeping for 1 seconds 
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1
Done 1
result 1


In [27]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [1,2,3,4,5]
    results = [executor.submit(do_something,sec) for sec in secs]
    for f in concurrent.futures.as_completed(results):
        print(f.result())

sleeping for 1 seconds 
sleeping for 2 seconds 
sleeping for 3 seconds 
sleeping for 4 seconds 
sleeping for 5 seconds 
Done 1
result 1
Done 2
result 2
Done 3
result 3
Done 4
result 4
Done 5
result 5


In [28]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = [executor.submit(do_something,sec) for sec in secs]
    for f in concurrent.futures.as_completed(results):
        print(f.result())
t2 = time.perf_counter()
print(t2-t1)

sleeping for 5 seconds 
sleeping for 4 seconds 
sleeping for 3 seconds 
sleeping for 2 seconds 
sleeping for 1 seconds 
Done 1
result 1
Done 2
result 2
Done 3
result 3
Done 4
result 4
Done 5
result 5
5.009151200007182


In [29]:
# using map function result will only be printed at the end , in the order in which inputs were fed in map function
# also while running map function, python doesn't raise an error if exceptions exists
# if exception exists, error will be raised while retrieving results using for loop

In [30]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = executor.map(do_something,secs) 
    for result in results:
        print(result)
t2 = time.perf_counter()
print(t2-t1)

sleeping for 5 seconds 
sleeping for 4 seconds 
sleeping for 3 seconds 
sleeping for 2 seconds 
sleeping for 1 seconds 
Done 1
Done 2
Done 3
Done 4
Done 5
result 5
result 4
result 3
result 2
result 1
5.00681920000352


In [31]:
# also you don't need to use join as well while using map func.
# it's because python will wait till all the secondary threads of map func are complete to continue running main thread below it